In [1]:
import numpy as np
import cv2
import os
import sys
import math
import random
import copy
import inspect
from matplotlib import pyplot as plot
import pickle


### une fonction pour Traite une image pour identifier et extraire la région de l'iris, détecte les points clés et calcule les descripteurs en utilisant d'autres fonctions auxiliaires.(fonction dapple )


In [ ]:

def extract_regions(image_path):
    image = read_image(image_path, display=False)

    print("Identifying iris boundaries...")
    inner_circle, outer_circle = locate_iris_boundaries(image, display=False)
    if not inner_circle or not outer_circle:
        print("Error locating iris boundaries!")
        return

    print("Equalizing histogram...")
    region = equalize_iris_region(image, outer_circle, inner_circle, display=False)

    print("Extracting iris region images...")
    regions = segment_regions(region, inner_circle, outer_circle, display=False)

    print("Detecting keypoints...")
    sift_engine = sift = cv2.SIFT_create()
    detect_keypoints(sift_engine, regions, display=False)
    compute_descriptors(sift_engine, regions)  

    return regions


Lit une image depuis un chemin donné et affiche l'image si demandé.


In [ ]:
def read_image(image_path, display=False):
    image = cv2.imread(image_path, 0)
    if display:
        cv2.imshow(image_path, image)
        char = cv2.waitKey(0)
        cv2.destroyAllWindows()
    return image

### Fonction : locate_iris_boundaries
Détecte dynamiquement les limites internes et externes de l'iris en ajustant progressivement les paramètres de recherche en fonction des résultats initiaux.
(La détection est effectuée dans d'autres fonctions. Cette fonction utilise les autres fonctions afin de détecter les bordures et ajuste les paramètres jusqu'à ce que les limites soient trouvées.)

In [ ]:
def locate_iris_boundaries(image, display=False):
    inner_circle = detect_inner_circle(image)

    if not inner_circle:
        print('ERROR: Inner circle not found!')
        return None, None

    radius_expansion = int(math.ceil(inner_circle[2]*1.5))
    expansion_factor = 0.25
    range_center = int(math.ceil(inner_circle[2]*expansion_factor)) 
    outer_circle = detect_outer_circle(
                        image, inner_circle, range_center, radius_expansion)

    while(not outer_circle and expansion_factor <= 0.7):
        expansion_factor += 0.05
        print('Searching outer iris circle with expansion factor ' + str(expansion_factor))

        range_center = int(math.ceil(inner_circle[2]*expansion_factor))
        outer_circle = detect_outer_circle(image, inner_circle,
                                        range_center, radius_expansion)
    if not outer_circle:
        print('ERROR: Outer iris circle not found!')
        return None, None
    
    if display:
        color_image = cv2.cvtColor(image,cv2.COLOR_GRAY2BGR)
        draw_detected_circles(color_image, inner_circle, outer_circle,
                     range_center, radius_expansion)
        cv2.imshow('iris boundaries', color_image)
        char = cv2.waitKey(0)
        cv2.destroyAllWindows()

    return inner_circle, outer_circle

### Fonction : detect_inner_circle
Détecte le cercle intérieur de l'iris en appliquant des filtres et en ajustant les seuils de manière dynamique pour optimiser la détection des cercles via la transformation de Hough.


In [ ]:
def detect_inner_circle(image):
    def extract_edges(processed_image):
        edges = cv2.Canny(processed_image, 20, 100)
        kernel = np.ones((3,3), np.uint8)
        edges = cv2.dilate(edges, kernel, iterations=2)
        blur_size = 2 * random.randrange(5, 11) + 1
        edges = cv2.GaussianBlur(edges, (blur_size, blur_size), 0)
        return edges

    circle_param1 = 200  # High threshold for Canny
    circle_param2 = 120  # Accumulator threshold for circle detection
    candidate_circles = []
    while(circle_param2 > 35 and len(candidate_circles) < 100):
        for median_filter_size, threshold in [(m, t) for m in [3, 5, 7] for t in [20, 25, 30, 35, 40, 45, 50, 55, 60]]:
            # Apply median blur
            blurred = cv2.medianBlur(image, 2 * median_filter_size + 1)

            # Apply threshold
            _, binary_image = cv2.threshold(blurred, threshold, 255, cv2.THRESH_BINARY_INV)

            # Fill contours
            contours, _ = cv2.findContours(binary_image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            filled_contours = cv2.drawContours(binary_image, contours, -1, (255), -1)

            # Extract edges
            edges = extract_edges(binary_image)

            # Detect circles using Hough transform
            circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 1, np.array([]), circle_param1, circle_param2)
            if circles is not None and circles.size > 0:
                # Convert the circle parameters to integers
                circles = np.round(circles[0, :]).astype("int")
                for circle in circles:
                    candidate_circles.append(circle)

        circle_param2 = circle_param2 - 1

    if len(candidate_circles) == 0:
        print("No inner circles detected.")
        return None

    return calculate_average_circle(candidate_circles)

### Fonction : calculate_average_circle
Calcule et retourne les coordonnées moyennes et le rayon moyen à partir d'une liste de cercles détectés.


In [ ]:

def calculate_average_circle(circles):
    if not circles:
        return
    average_x = int(np.mean([c[0] for c in circles]))
    average_y = int(np.mean([c[1] for c in circles]))
    average_radius = int(np.mean([c[2] for c in circles]))

    return average_x, average_y, average_radius

### Fonction : identify_optimal_radius
Identifie le rayon optimal en minimisant la distance totale par rapport aux autres rayons dans la liste, permettant une sélection plus précise du cercle.


In [ ]:
def identify_optimal_radius(circle_list):
    optimal_circle = None
    minimum_distance = None
    reference_circles = circle_list[:]
    comparison_circles = circle_list[:]
    for current_circle in reference_circles:
        total_distance = 0
        for compared_circle in comparison_circles:
            total_distance += math.fabs(float(current_circle[2]) - float(compared_circle[2]))
        if not minimum_distance or total_distance < minimum_distance:
            minimum_distance = total_distance
            optimal_circle = current_circle
    return optimal_circle[2]


### Fonction : detect_outer_circle
Détecte dynamiquement le cercle extérieur de l'iris en analysant les bords de l'image traitée. La fonction utilise plusieurs tailles de flou médian et des seuils de détection ajustés itérativement pour identifier les cercles par transformation de Hough. La détection continue jusqu'à ce que le nombre cible de cercles soit atteint ou que les seuils de détection soient épuisés, tout en vérifiant que les cercles détectés se trouvent bien à l'extérieur du cercle intérieur mais dans les limites spécifiées.

In [ ]:

def detect_outer_circle(image, inner_circle, center_range, radius_range):
    def extract_edges(processed_image, upper_threshold):
        lower_threshold = 0  # Fixed low threshold for Canny
        edges = cv2.Canny(processed_image, lower_threshold, upper_threshold, apertureSize=5)
        kernel = np.ones((3,3),np.uint8)
        edges = cv2.dilate(edges, kernel, iterations=1)
        blur_size = 2 * random.randrange(5,11) + 1
        edges = cv2.GaussianBlur(edges,(blur_size,blur_size),0)
        return edges

    def find_circles(hough_threshold, median_sizes, edge_thresholds):
        detected_circles = []
        for median_size, upper_threshold in [(m, t) for m in median_sizes for t in edge_thresholds]:
            # Apply median blur
            median_blurred = cv2.medianBlur(image, 2 * median_size + 1)

            # Extract edges
            edges = extract_edges(median_blurred, upper_threshold)

            # Detect circles using Hough transform
            circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 20,
                                       param1=200, param2=hough_threshold, minRadius=0, maxRadius=0)
            if circles is not None and circles.size > 0:
                # Convert the circle parameters to integers
                circles = np.round(circles[0, :]).astype("int")
                for (col, row, radius) in circles:
                    if within_circle(inner_circle[0], inner_circle[1], center_range, col, row) and radius > radius_range:
                        detected_circles.append((col, row, radius))
        return detected_circles

    circle_param2 = 120  # Starting threshold for HoughCircles
    all_circles = []
    while(circle_param2 > 40 and len(all_circles) < 50):
        circles = find_circles(
                        circle_param2, [8,10,12,14,16,18,20], [430,480,530])
        if circles:
            all_circles += circles
        circle_param2 = circle_param2 -1

    if not all_circles:
        print("Alternative strategy for detecting outer iris circle")
        circle_param2 = 120
        while(circle_param2 > 40 and len(all_circles) < 50):
            circles = find_circles(
                            circle_param2, [3,5,7,21,23,25], [430,480,530])
            if circles:
                all_circles += circles
            circle_param2 = circle_param2 -1

    if not all_circles:
        return

    color_image = cv2.cvtColor(image,cv2.COLOR_GRAY2BGR)
    refined_circles = refine_circle_selection(all_circles)

    return calculate_average_circle(refined_circles)

### Fonction : within_circle
Vérifie si un point donné se trouve à l'intérieur du rayon d'un cercle spécifié, en utilisant la distance entre le centre du cercle et le point.


In [ ]:

def within_circle(circle_col, circle_row, circle_radius, point_col, point_row):
    return compute_distance(circle_col, circle_row, point_col, point_row) <= circle_radius

### Fonction : refine_circle_selection
Affine la sélection de cercles détectés en filtrant ceux qui s'écartent significativement des moyennes calculées pour les positions et rayons. cet fonction Utilise des écarts types pour définir les seuils d'acceptation, en éliminant les cercles qui ne correspondent pas aux critères statistiques de cohérence des données.


In [ ]:
def refine_circle_selection(circles):
    if not circles:
        print('Error: No circles found in refine_circle_selection() !')
        return []
    center_x_mean, center_x_dev = compute_standard_deviation([int(i[0]) for i in circles])
    center_y_mean, center_y_dev = compute_standard_deviation([int(i[1]) for i in circles])
    refined = []
    positions = []
    unrefined = []
    deviation_multiplier = 1.5 
    for circle in circles[:]:
        if circle[0] < center_x_mean - deviation_multiplier*center_x_dev or \
           circle[0] > center_x_mean + deviation_multiplier*center_x_dev or \
           circle[1] < center_y_mean - deviation_multiplier*center_y_dev or \
           circle[1] > center_y_mean + deviation_multiplier*center_y_dev:
            unrefined.append(circle)
        else:
            positions.append(circle)
    if len([float(c[2]) for c in positions]) < 3:
        refined = positions
    else:
        optimal_radius = identify_optimal_radius(positions)
        mean_radius, radius_deviation = compute_standard_deviation(
                                    [float(c[2]) for c in positions])
        max_radius = optimal_radius + radius_deviation
        min_radius = optimal_radius - radius_deviation
        for circle in positions:
            if circle[2] < min_radius or \
               circle[2] > max_radius:
                unrefined.append(circle)
            else:
                refined.append(circle)

    return refined

### Fonction : draw_detected_circles
Dessine les cercles détectés sur une image colorée, y compris les cercles internes et externes de l'iris, ainsi que les limites spécifiques de portée et de rayon si elles sont fournies.


In [ ]:

def draw_detected_circles(colored_image, inner_circle, outer_circle,
                 center_range=None, radius_range=None):
    # draw the inner circle
    cv2.circle(colored_image,(inner_circle[0], inner_circle[1]), inner_circle[2],
                     (0,0,255),1)
    # draw the center of the inner circle
    cv2.circle(colored_image,(inner_circle[0],inner_circle[1]),1,(0,0,255),1)
    if center_range:
        # draw outer circle center range limit
        cv2.circle(colored_image,(inner_circle[0], inner_circle[1]), center_range,
                         (0,255,255),1)
    if radius_range:
        # draw outer circle radius range limit
        cv2.circle(colored_image,(inner_circle[0], inner_circle[1]), radius_range,
                         (0,255,255),1)
    # draw the outer circle
    cv2.circle(colored_image, (outer_circle[0], outer_circle[1]), 
               outer_circle[2],(0,255,0),1)
    # draw the center of the outer circle
    cv2.circle(colored_image, (outer_circle[0], outer_circle[1]), 
               1,(0,255,0),1)
    

### Fonction : equalize_iris_region
Égalise l'histogramme d'une région spécifiée de l'iris, qui est délimitée par les cercles intérieur et extérieur, en masquant et en ajustant dynamiquement les niveaux d'intensité pour améliorer la visibilité et le contraste.


In [ ]:
   
def equalize_iris_region(image, outer_circle, inner_circle, display=False):
    def locate_roi():
        mask = image.copy()
        mask[:] = (0)

        cv2.circle(mask, 
                   (outer_circle[0], outer_circle[1]), 
                   outer_circle[2], (255), -1)
        cv2.circle(mask,
                   (inner_circle[0],inner_circle[1]),
                   inner_circle[2],(0), -1)

        region_of_interest = cv2.bitwise_and(image, mask)

        return region_of_interest

    region_of_interest = locate_roi()

    # Mask the top part of the iris
    for column_index in range(region_of_interest.shape[1]):
        for row_index in range(region_of_interest.shape[0]):
            angle = calculate_angle(outer_circle[0], outer_circle[1], 
                            column_index, row_index)
            if angle > 50 and angle < 130:
                region_of_interest[row_index,column_index] = 0

    _, region_of_interest = cv2.threshold(region_of_interest,50,255,cv2.THRESH_TOZERO)

    equalized_region = region_of_interest.copy()
    cv2.equalizeHist(region_of_interest, equalized_region)
    region_of_interest = cv2.addWeighted(region_of_interest, 0.0, equalized_region, 1.0, 0)

    if display:
        cv2.imshow('equalized histogram iris region', region_of_interest)
        char = cv2.waitKey(0)
        cv2.destroyAllWindows()

    return region_of_interest

### Fonction : segment_regions
Segmente et isole la région de l'iris entre les cercles intérieur et extérieur pour un traitement ultérieur. Cette fonction ajuste également la position de la région segmentée pour centraliser les cercles de l'iris, appliquant une transformation pour aligner les images selon les nouveaux centres calculés.


In [ ]:
    
def segment_regions(image, inner_circle, outer_circle, display=False):
    background = image.copy()
    background[:] = 0

    regions = {'entire': {'image': background.copy(),
                          'inner_circle': inner_circle,
                          'outer_circle': outer_circle,
                          'keypoints': None,
                          'image_with_keypoints': background.copy(),
                          'image_with_filtered_keypoints': background.copy(),
                          'descriptors': None
                         }}

    for column_index in range(image.shape[1]):
        for row_index in range(image.shape[0]):
            if not within_circle(inner_circle[0], inner_circle[1], inner_circle[2], column_index, row_index) and \
               within_circle(outer_circle[0], outer_circle[1], outer_circle[2], column_index, row_index):
                regions['entire']['image'][row_index,column_index] = image[row_index,column_index]

    regions['entire']['outer_circle'] = (int(1.25*outer_circle[2]), 
                                         int(1.25*outer_circle[2]),
                                         int(outer_circle[2]))
    translate_x = regions['entire']['outer_circle'][0] - outer_circle[0]
    translate_y = regions['entire']['outer_circle'][1] - outer_circle[1]
    regions['entire']['inner_circle'] = (int(translate_x + inner_circle[0]),
                                         int(translate_y + inner_circle[1]),
                                         int(inner_circle[2]))

    translation_matrix = np.float32([[1,0,translate_x],[0,1,translate_y]])
    regions['entire']['image'] = cv2.warpAffine(
                        regions['entire']['image'], translation_matrix, 
                        (image.shape[1], image.shape[0]))

    regions['entire']['image'] = regions['entire']['image'][0:int(2.5 * outer_circle[2]), 0:int(2.5 * outer_circle[2])]

    if display:
        plot.imshow(regions['entire']['image'], cmap='gray')
        plot.title('Entire'), plot.xticks([]), plot.yticks([])
        plot.show()

    return regions

### Fonction : detect_keypoints
Détecte les points clés de la région de l'iris en utilisant l'algorithme SIFT, en excluant les points en dehors de la région spécifiée entre les cercles intérieur et extérieur. Les points clés valides sont visualisés sur l'image si demandé.


In [ ]:

def detect_keypoints(sift_engine, regions, display=False):    
    all_keypoints = sift_engine.detect(regions['entire']['image'], None)
    valid_keypoints = []
    for keypoint in all_keypoints:
        x, y = keypoint.pt
        if within_circle(regions['entire']['outer_circle'][0], regions['entire']['outer_circle'][1], regions['entire']['outer_circle'][2], x, y) and \
           not within_circle(regions['entire']['inner_circle'][0], regions['entire']['inner_circle'][1], regions['entire']['inner_circle'][2], x, y):
            valid_keypoints.append(keypoint)
    regions['entire']['keypoints'] = valid_keypoints

    regions['entire']['image_with_keypoints'] = cv2.drawKeypoints(
                                    regions['entire']['image'], regions['entire']['keypoints'],
                                    color=(0,255,0), flags=0,
                                    outImage=None)
    if display:
        plot.imshow(cv2.cvtColor(regions['entire']['image_with_keypoints'], cv2.COLOR_BGR2RGB))
        plot.title('Entire Region with KeyPoints')
        plot.axis('off')
        plot.show()

### Fonction : compute_descriptors
Calcule les descripteurs pour les points clés détectés dans la région de l'iris en utilisant l'engine SIFT. Affiche les dimensions des descripteurs si disponibles, signalant également quand aucun descripteur n'est trouvé.


In [ ]:

def compute_descriptors(sift_engine, regions):
    regions['entire']['keypoints'], regions['entire']['descriptors'] = sift_engine.compute(regions['entire']['image'], regions['entire']['keypoints'])
    if regions['entire']['descriptors'] is not None:
        print(f"Entire Region, Descriptor Dimensions: {regions['entire']['descriptors'].shape}")
    else:
        print("Entire Region, No descriptors found.")



### Fonction : match_all_regions
Orchestre la comparaison des régions entre deux images en utilisant la fonction `find_matches` pour obtenir les correspondances. Visualise ces correspondances et indique leur nombre total.

In [ ]:

def match_all_regions(regions_one, regions_two, ratio_threshold, angle_standard_deviation, distance_standard_deviation, display=False):
    if display:
        plot.figure(figsize=(10, 5))
        plot.title('Keypoint Matches')

    if not regions_one['entire']['keypoints'] or not regions_two['entire']['keypoints']:
        print("KeyPoints not found in one or both images for entire region!!!")
        return 0

    matches = find_matches(regions_one['entire'], regions_two['entire'], ratio_threshold, angle_standard_deviation, distance_standard_deviation)
    if display:
        if matches:
            matches_to_draw = [[m] for m in matches]
            matched_image = cv2.drawMatchesKnn(
                regions_one['entire']['image'], regions_one['entire']['keypoints'],
                regions_two['entire']['image'], regions_two['entire']['keypoints'],
                matches_to_draw, None, flags=2)
            matched_image_rgb = cv2.cvtColor(matched_image, cv2.COLOR_BGR2RGB)
            plot.imshow(matched_image_rgb)
            plot.title(f'Matches in Entire Region')
            plot.axis('off')
        else:
            print("No matches to draw for entire region")

        plot.show()
    print(f"Total matches found: {len(matches)}") 

    return len(matches)

### Fonction : find_matches
Filtre les correspondances entre les points clés des deux régions en évaluant la cohérence géométrique basée sur des seuils de ratio, d'angle et de distance. Cette analyse détaillée assure que seules les correspondances les plus fiables sont conservées, utilisant des critères précis pour évaluer la similitude des points clés.


In [ ]:

def find_matches(region_one, region_two, ratio_threshold, angle_deviation_threshold, distance_deviation_threshold):    
    if not region_one['keypoints'] or not region_two['keypoints']:
        print("KeyPoints not found in one of region_x['keypoints'] !!!")
        return []

    matcher = cv2.BFMatcher()
    raw_matches = matcher.knnMatch(region_one['descriptors'], region_two['descriptors'], k=2)
    print(f"Raw matches found: {len(raw_matches)}")
    kp1 = region_one['keypoints']
    kp2 = region_two['keypoints']

    distance_diff_one = region_one['outer_circle'][2] - region_one['inner_circle'][2]
    distance_diff_two = region_two['outer_circle'][2] - region_two['inner_circle'][2]

    angle_differences = []
    distance_differences = []
    filtered_matches = []
    for m,n in raw_matches:
        if (m.distance/n.distance) > ratio_threshold:
            continue
        
        x1,y1 = kp1[m.queryIdx].pt
        x2,y2 = kp2[m.trainIdx].pt

        angle_one = calculate_angle(
                x1,y1,
                region_one['inner_circle'][0],
                region_one['inner_circle'][1])
        angle_two = calculate_angle(
                x2,y2,
                region_two['inner_circle'][0],
                region_two['inner_circle'][1])
        angle_difference = angle_one - angle_two
        angle_differences.append(angle_difference)

        distance_one = compute_distance(x1,y1,
                          region_one['inner_circle'][0],
                          region_one['inner_circle'][1])
        distance_one = distance_one - region_one['inner_circle'][2]
        distance_one = distance_one / distance_diff_one
        
        distance_two = compute_distance(x2,y2,
                          region_two['inner_circle'][0],
                          region_two['inner_circle'][1])
        distance_two = distance_two - region_two['inner_circle'][2]
        distance_two = distance_two / distance_diff_two

        distance_difference = distance_one - distance_two
        distance_differences.append(distance_difference)
        
        filtered_matches.append(m)

    if filtered_matches:
        median_angle_difference = compute_median(angle_differences)
        median_distance_difference = compute_median(distance_differences)
        for match in filtered_matches[:]:
            x1,y1 = kp1[match.queryIdx].pt
            x2,y2 = kp2[match.trainIdx].pt

            angle_one = calculate_angle(
                x1,y1,
                region_one['inner_circle'][0],
                region_one['inner_circle'][1])
            angle_two = calculate_angle(
                x2,y2,
                region_two['inner_circle'][0],
                region_two['inner_circle'][1])
            angle_difference = angle_one - angle_two

            within_angle_limits = \
                (angle_difference > median_angle_difference - angle_deviation_threshold and \
                 angle_difference < median_angle_difference + angle_deviation_threshold)

            distance_one = compute_distance(x1,y1,
                              region_one['inner_circle'][0],
                              region_one['inner_circle'][1])
            distance_one = distance_one - region_one['inner_circle'][2]
            distance_one = distance_one / distance_diff_one
        
            distance_two = compute_distance(x2,y2,
                              region_two['inner_circle'][0],
                              region_two['inner_circle'][1])
            distance_two = distance_two - region_two['inner_circle'][2]
            distance_two = distance_two / distance_diff_two

            distance_difference = distance_one - distance_two
            within_distance_limits = (distance_difference > median_distance_difference - distance_deviation_threshold and \
                         distance_difference < median_distance_difference + distance_deviation_threshold)
                
            if within_angle_limits and within_distance_limits:
                continue

            filtered_matches.remove(match)

    return filtered_matches

###  Fonctions Mathématiques Utiles
Ces fonctions fournissent des calculs essentiels pour l'analyse et le traitement d'images:

- **calculate_angle(x1, y1, x2, y2):** Calcule l'angle en degrés entre deux points.
- **compute_distance(x1, y1, x2, y2):** Détermine la distance euclidienne entre deux points.
- **compute_mean(values):** Calcule la moyenne d'une liste de valeurs numériques.
- **compute_median(values):** Trouve la valeur médiane d'une liste, offrant une mesure centrale robuste.
- **compute_standard_deviation(values):** Calcule l'écart type pour mesurer la variation ou la dispersion des valeurs autour de la moyenne.

Ces outils sont cruciaux pour les opérations géométriques et l'analyse statistique dans le traitement des données d'image.


In [ ]:

def calculate_angle(x1, y1, x2, y2):
    return math.degrees(math.atan2(-(y2-y1),(x2-x1)))

def compute_distance(x1, y1, x2, y2):
    distance = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

def compute_mean(values):
    total = 0.0
    for value in values:
        total += value
    return total/len(values)

def compute_median(values):
    return np.median(np.array(values))

def compute_standard_deviation(values):
    if not values:
        print('Error: empty list parameter in compute_standard_deviation() !')
        return None, None
    mean_value = compute_mean(values)
    sum_of_squares = 0.0
    for value in values:
        sum_of_squares += (value - mean_value) ** 2
    return mean_value, math.sqrt(sum_of_squares/len(values))


### Fonctions : serialize_keypoints et deserialize_keypoints
Ces fonctions gèrent la sérialisation et la désérialisation des points clés utilisés dans les algorithmes SIFT pour faciliter le stockage et la comparaison:

- **serialize_keypoints(keypoints):** Convertit une liste de points clés en une structure de données sérialisée, préparant les données pour un stockage efficace dans un fichier pickle.
- **deserialize_keypoints(serialized_keypoints):** Reconstruit les points clés à partir de leur forme sérialisée, permettant la comparaison des données d'image stockées avec une nouvelle image test pour la reconnaissance diris.

Ces fonctions sont essentielles pour crypter les points de données de SIFT lors de l'enregistrement dans une base de données et pour décrypter ces points lors de la comparaison avec une nouvelle image.


In [ ]:

def serialize_keypoints(keypoints):
    return [{'pt': keypoint.pt, 'size': keypoint.size, 'angle': keypoint.angle,
             'response': keypoint.response, 'octave': keypoint.octave, 'class_id': keypoint.class_id}
            for keypoint in keypoints]
def deserialize_keypoints(serialized_keypoints):
    return [cv2.KeyPoint(x=kp['pt'][0], y=kp['pt'][1], size=kp['size'], angle=kp['angle'],
                         response=kp['response'], octave=kp['octave'], class_id=kp['class_id'])
            for kp in serialized_keypoints]


###  Fonction : inspect_region_information
Affiche des informations essentielles sur une région d'image traitée, notamment les dimensions de l'image et le nombre de points clés détectés. Cette fonction facilite le diagnostic rapide de l'état des données de région après le traitement, ce qui est crucial pour le débogage et l'analyse de performance.


In [ ]:

def inspect_region_information(regions):
    data = regions['entire']
    print("Region: Entire")
    print(f"Image dimensions: {data['image'].shape}")
    print(f"Number of keypoints: {len(data['keypoints'])}")

### Fonction : preprocess_and_serialize
Crée une base de données sérialisée pour la reconnaissance d'images à partir d'un chemin de données spécifié. Pour chaque individu, la fonction sélectionne la première image, extrait les régions pertinentes de l'iris, sérialise les points clés et enregistre les données sérialisées dans un fichier pickle dans le répertoire de sortie. Cela permet de préparer les données nécessaires pour la reconnaissance ultérieure.

**Paramètres :**
- `dataset_path`: Chemin vers le répertoire contenant les images brutes.
- `output_path`: Chemin vers le répertoire où les données sérialisées seront sauvegardées.

**Étapes :**
1. Parcourt chaque dossier d'individu dans le répertoire de données.
2. Sélectionne la première image de chaque dossier.
3. Extrait les régions de l'iris et sérialise les points clés.
4. Sauvegarde les données sérialisées dans un fichier pickle.


In [ ]:
def preprocess_and_serialize(dataset_path, output_path):
    for person_id in sorted(os.listdir(dataset_path)):
        person_path = os.path.join(dataset_path, person_id)
        if os.path.isdir(person_path):
            images = sorted(os.listdir(person_path))
            if images:  
                image_name = images[0]  
                image_path = os.path.join(person_path, image_name)
                region_data = extract_regions(image_path)
                if region_data:
                    region_data['entire']['keypoints'] = serialize_keypoints(region_data['entire']['keypoints'])
                    serialized_file_path = os.path.join(output_path, f"{person_id}_{image_name}.pkl")
                    with open(serialized_file_path, 'wb') as f:
                        pickle.dump(region_data, f)
                        print(f"Serialized data for {person_id} saved to {serialized_file_path}")

In [ ]:
# Example usage
dataset_path = 'A_Data\\Raw_Data'
output_path = 'B_SIFT_knn_bfm\\3_DATA_modele_data_base'

#preprocess_and_serialize(dataset_path, output_path)


serialized_data_path ='C:\\Users\\pc\\Desktop\\Backend\\B_SIFT_knn_bfm\\3_DATA_modele_data_base'
test_image_path = 'C:\\Users\\pc\\Desktop\\Backend\\A_Data\\Raw_Data\\001\\S6001S02.jpg'
raw_data = 'C:\\Users\\pc\\Desktop\\Backend\\A_Data\\Raw_Data'

### Fonction pour effectuer une comparaison visuelle entre l'image test et l'image de la classe prédite

Cette fonction, `perform_visual_comparison`, est conçue pour valider visuellement la précision de la prédiction du modèle en comparant limage  avec une image de référence de la classe prédite, en utilisant des données sérialisées.


In [ ]:
def perform_visual_comparison(predicted_class, raw_data_path, serialized_data_path, test_regions):
    # Load the first image from the predicted class's raw data folder
    predicted_class_folder = os.path.join(raw_data_path, predicted_class)
    predicted_image_path = os.path.join(predicted_class_folder, os.listdir(predicted_class_folder)[0])
    
    # Deserialize the predicted class structure
    for file in os.listdir(serialized_data_path):
        if file.startswith(predicted_class + "_"):
            serialized_file_path = os.path.join(serialized_data_path, file)
            break

    with open(serialized_file_path, 'rb') as f:
        serialized_data = pickle.load(f)
    serialized_data['entire']['keypoints'] = deserialize_keypoints(serialized_data['entire']['keypoints'])

    # Use the deserialized keypoints and descriptors to match against the test image
    match_all_regions(test_regions, serialized_data, 0.8, 10, 0.15, display=True)



### Fonction : find_best_match
Compare une image de test avec des données sérialisées stockées pour identifier la meilleure correspondance. Cette fonction extrait les régions de l'image de test, désérialise les données de points clés pour chaque entrée de la base de données, et utilise `match_all_regions` pour compter et comparer les correspondances. Elle rapporte la meilleure correspondance en fonction du nombre maximum de correspondances trouvées, facilitant ainsi la reconnaissance ou la vérification d'identité.
et cette fonction utilise perform_visual_comparison pour visualiser la comparaison

In [ ]:

def find_best_match(test_image_path, serialized_data_path):
    test_data = extract_regions(test_image_path)


    max_matches = 0
    best_match = None
        
    for serialized_file in os.listdir(serialized_data_path):
            with open(os.path.join(serialized_data_path, serialized_file), 'rb') as f:
                person_data = pickle.load(f)
                person_data['entire']['keypoints'] = deserialize_keypoints(person_data['entire']['keypoints'])
                matches = match_all_regions(test_data, person_data, 0.8, 10, 0.15, display=False)
                
                if matches > max_matches:
                    max_matches = matches
                    best_match = serialized_file
        
    if best_match:
            best_match = best_match.split('_')[0]
            perform_visual_comparison(best_match, raw_data, serialized_data_path, test_data)

            print(f"The test image matches best with {best_match} with {max_matches} matches.")
            return best_match

    else:
            print("No match found.")


### tester une seul image

In [ ]:


serialized_data_path ='B_SIFT_knn_bfm\\3_DATA_modele_data_base'
test_image_path = 'A_Data\\Raw_Data\\001\\S6001S02.jpg'

In [ ]:
#id = find_best_match(test_image_path, serialized_data_path)


### Fonction : testing_and_accuracy_calculation
Cette fonction teste de nouvelles images qui n'étaient pas sauvegardées dans la base de données pour vérifier leur précision(classes) et la precision totale. Après le test, la précision était de 98 %, où chaque personne avait 1 image traitée et cryptée dans la base de données, et 9 images étaient utilisées pour les tests.

In [ ]:


def testing_and_accuracy_calculation(dataset_path,base):
    i = [0] * 100 
    j=0
    accuracies = []
    total_correct_matches = 0
    total_images = 0

    for person_id in sorted(os.listdir(dataset_path)):
        person_path = os.path.join(dataset_path, person_id)
        if os.path.isdir(person_path):
            images = sorted(os.listdir(person_path))
            if images:
                for image in images[1:]:  
                  image_name = image  
                  image_path = os.path.join(person_path, image_name)
                  matched_id = find_best_match(image_path, base)
                  if matched_id == person_id:
                     i[j]=i[j]+1
                     continue
            correct_matches = i[j]
            total_correct_matches += correct_matches
            total_images += 9

            accuracy = correct_matches /9
            accuracies.append((person_id, accuracy))
        j=j+1   

    overall_accuracy = total_correct_matches / total_images if total_images > 0 else 0
    for person_id, accuracy in accuracies:
        print(f"Person {person_id}: Accuracy = {accuracy:.2%}")
    print(f"Overall Accuracy = {overall_accuracy:.2%}")
    

               
#testing_and_accuracy_calculation('A_Data\\Raw_Data','B_SIFT_knn_bfm\\3_DATA_modele_data_base')      